# Galaxy Pair Selection

Implementation of selection criteria for galaxy pairs:
1. Maximum separation of 50 kpc projected
2. Similar stellar masses
3. Velocity difference < 300 km/s
4. Star-forming galaxies
5. Size constraint for NIRSpec FoV
6. Halpha observability

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
from astropy import units as u
import warnings
warnings.filterwarnings('ignore')

# Define cosmology (Planck 2018)
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)

# Import our analysis functions
import sys
sys.path.append('../scripts/analysis')
from catalog_analysis import load_selection_data, analyze_data_quality, check_nirspec_fov_constraints

## Apply Selection Criteria

In [2]:
# Load data
print("=== LOADING DATA ===")
data_path = "../data/raw/selection_criteria_data.csv"

# Try to load data with proper error handling
try:
    catalog = pd.read_csv(data_path)
    print(f"✓ Catalog loaded successfully: {len(catalog):,} galaxies")
    print("Available columns:", list(catalog.columns))
    
    # Show basic info
    print(f"\nData shape: {catalog.shape}")
    print("\nFirst few rows:")
    print(catalog.head())
    
except FileNotFoundError:
    print(f"❌ File not found: {data_path}")
    print("Please check if the data file exists at this location")
    catalog = None
except Exception as e:
    print(f"❌ Error loading data: {e}")
    catalog = None

=== LOADING DATA ===
✓ Catalog loaded successfully: 12,129 galaxies
Available columns: ['id', 'ra', 'dec', 'z', 'zfinal', 'logm', 'sfr', 'sfr_inst', 'disk_radius_deg', 'bulge_radius_deg', 'disk_axratio', 'bulge_axratio', 'Re_kpc', 'incl_deg', 'morph_flag_f444w', 'irr_f444w_mean', 'disk_f444w_mean', 'logSFR', 'logSFR_MS', 'DeltaMS', 'selected_as']

Data shape: (12129, 21)

First few rows:
    id          ra       dec       z  zfinal       logm        sfr   sfr_inst   
0   78  149.733127  2.150583  2.5558  2.5558   9.883062  13.029404  13.029404  \
1  108  149.815323  2.120871  2.2696  2.2696  10.023751  35.704725  35.704725   
2  230  149.861690  2.105282  2.3527  2.3527   9.914175  77.251815  77.251815   
3  231  149.742434  2.148731  2.3928  2.3928   9.980232  13.691141  13.691141   
4  285  149.758131  2.142487  1.6294  1.6294  10.310997  18.034745  18.034745   

   disk_radius_deg  bulge_radius_deg  ...  bulge_axratio  Re_kpc   incl_deg   
0         0.000029      2.588507e-05  ...  

In [3]:
# Initialize variables to avoid "not defined" errors
filtered_catalog = catalog.copy()
selected_pairs = []
control_sample = pd.DataFrame()

print("✓ Variables initialized")

✓ Variables initialized


In [4]:
def main_sequence_logSFR(logm, z):
    """
    Calculate main sequence SFR based on Speagle+2014 relation.
    Improved version from test_pairs notebook.
    
    Parameters:
    -----------
    logm : float or array
        Log stellar mass
    z : float or array
        Redshift
        
    Returns:
    --------
    float or array
        Log SFR on the main sequence
    """
    a = 0.84 - 0.026*z
    b = (0.11*z - 6.51) + 0.7
    return a*(logm-10.5) + b

def calculate_projected_separation_optimized(ra1, dec1, ra2, dec2, z_avg):
    """
    Optimized version using astropy SkyCoord for better accuracy.
    From test_pairs improvements.
    """
    # Create SkyCoord objects
    coord1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree, frame='icrs')
    coord2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree, frame='icrs')
    
    # Calculate angular separation
    angular_sep = coord1.separation(coord2)
    
    # Convert to physical separation using cosmology
    angular_diameter_distance = cosmo.angular_diameter_distance(z_avg).to(u.kpc).value
    physical_sep_kpc = angular_sep.radian * angular_diameter_distance
    
    return physical_sep_kpc

def check_mass_similarity(mass1, mass2, tolerance=0.3):
    """Check if two galaxies have similar stellar masses (within tolerance in dex)."""
    mass_diff = abs(mass1 - mass2)
    return mass_diff <= tolerance

def check_velocity_difference(z1, z2, max_diff_kms=300):
    """
    Check if velocity difference is less than max_diff km/s.
    Improved version with better velocity calculation.
    """
    c_kms = 299792.458  # km/s
    
    # Convert redshifts to velocities (Hubble flow approximation)
    v1 = z1 * c_kms
    v2 = z2 * c_kms
    
    delta_v = abs(v1 - v2)
    
    return delta_v <= max_diff_kms, delta_v

def check_star_forming_criterion(deltams):
    """Check if galaxy is star-forming (on/above main sequence)."""
    return deltams >= 0.0

def check_size_constraint_nirspec(z, re_kpc, fov_arcsec=3.0, n_re=2):
    """Check if 2*R_eff fits within NIRSpec IFU FoV."""
    # Convert effective radius to arcsec
    angular_diameter_distance = cosmo.angular_diameter_distance(z).to(u.kpc).value
    re_arcsec = (re_kpc / angular_diameter_distance) * 206265  # arcsec
    
    # Check if n_re * Re fits within FoV
    diameter_needed = n_re * re_arcsec
    return diameter_needed <= fov_arcsec, re_arcsec, diameter_needed

def check_halpha_observable(z, lambda_min=0.97, lambda_max=5.27, lambda_halpha=0.6563):
    """
    Check if Halpha line is observable with NIRSpec at given redshift.
    """
    # Observed wavelength of Halpha
    lambda_obs = lambda_halpha * (1 + z)
    
    return (lambda_min <= lambda_obs <= lambda_max), lambda_obs

print("✓ Optimized selection functions defined (with test_pairs improvements)")

✓ Optimized selection functions defined (with test_pairs improvements)


In [5]:
def find_galaxy_pairs_optimized(catalog, max_separation_kpc=50, max_velocity_diff=300):
    """
    Optimized galaxy pair finding algorithm using vectorized operations
    """
    from astropy.cosmology import Planck18 as cosmo
    from astropy.coordinates import SkyCoord
    import astropy.units as u
    
    # Convert coordinates to SkyCoord for efficient angular separation calculations
    coords = SkyCoord(ra=catalog['ra'].values*u.degree, 
                     dec=catalog['dec'].values*u.degree, 
                     frame='icrs')
    
    # Initialize lists to store pair indices
    galaxy_pairs = []
    
    # Calculate pairs using vectorized operations where possible
    n_galaxies = len(catalog)
    
    for i in range(n_galaxies - 1):
        # Get redshift and position for galaxy i
        z_i = catalog.iloc[i]['zfinal']
        coord_i = coords[i]
        
        # Calculate angular separations to all remaining galaxies
        remaining_coords = coords[i+1:]
        separations = coord_i.separation(remaining_coords)
        
        # Calculate angular diameter distance for galaxy i
        angular_diameter_distance = cosmo.angular_diameter_distance(z_i).to(u.kpc).value
        
        # Convert angular separations to physical separations
        physical_separations = separations.radian * angular_diameter_distance
        
        # Get redshifts for remaining galaxies
        remaining_redshifts = catalog.iloc[i+1:]['zfinal'].values
        
        # Calculate velocity differences
        c = 299792.458  # Speed of light in km/s
        velocity_i = z_i * c
        velocity_remaining = remaining_redshifts * c
        velocity_differences = np.abs(velocity_i - velocity_remaining)
        
        # Find valid pairs based on separation and velocity criteria
        valid_mask = (physical_separations < max_separation_kpc) & (velocity_differences < max_velocity_diff)
        valid_indices = np.where(valid_mask)[0]
        
        # Store valid pairs (adjust indices for the remaining galaxies)
        for j_offset in valid_indices:
            j = i + 1 + j_offset
            galaxy_pairs.append((i, j))
    
    return galaxy_pairs

In [6]:
# Apply optimized pair selection
print("Finding galaxy pairs with optimized algorithm...")
potential_pairs = find_galaxy_pairs_optimized(
    filtered_catalog,
    max_separation_kpc=50,
    max_velocity_diff=300
)

print(f"Found {len(potential_pairs)} potential galaxy pairs")

# Select first 10 pairs that meet all criteria
selected_pairs = []
pair_info = []

for i, (idx1, idx2) in enumerate(potential_pairs):
    if len(selected_pairs) >= 10:
        break
    
    # Get galaxy data
    galaxy1 = filtered_catalog.iloc[idx1]
    galaxy2 = filtered_catalog.iloc[idx2]
    
    # Calculate physical separation using proper cosmology
    coord1 = SkyCoord(ra=galaxy1['ra']*u.degree, dec=galaxy1['dec']*u.degree, frame='icrs')
    coord2 = SkyCoord(ra=galaxy2['ra']*u.degree, dec=galaxy2['dec']*u.degree, frame='icrs')
    angular_sep = coord1.separation(coord2)
    
    # Use average redshift for distance calculation
    z_avg = (galaxy1['zfinal'] + galaxy2['zfinal']) / 2
    angular_diameter_distance = cosmo.angular_diameter_distance(z_avg).to(u.kpc).value
    physical_sep_kpc = angular_sep.radian * angular_diameter_distance
    
    # Check mass similarity (within 0.3 dex)
    mass_diff = abs(galaxy1['logm'] - galaxy2['logm'])
    if mass_diff > 0.3:
        continue
    
    # Check if both galaxies are star-forming (Δ_MS ≥ 0)
    if galaxy1['DeltaMS'] < 0 or galaxy2['DeltaMS'] < 0:
        continue
    
    # Check NIRSpec FoV constraint (2×Re ≤ 3 arcsec)
    # Calculate angular size for both galaxies
    for galaxy in [galaxy1, galaxy2]:
        z = galaxy['zfinal']
        if pd.isna(galaxy['Re_kpc']):
            continue
            
        # Convert Re to arcsec
        angular_diameter_distance = cosmo.angular_diameter_distance(z).to(u.kpc).value
        re_arcsec = (galaxy['Re_kpc'] / angular_diameter_distance) * 206265  # Convert to arcsec
        if 2 * re_arcsec > 3.0:  # NIRSpec FoV is 3 arcsec
            continue
    
    # Check Hα observability (0.97-5.27 μm)
    lambda_halpha = 0.6563  # microns
    for galaxy in [galaxy1, galaxy2]:
        z = galaxy['zfinal']
        lambda_obs = lambda_halpha * (1 + z)
        if not (0.97 <= lambda_obs <= 5.27):
            continue
    
    # Add to selected pairs
    selected_pairs.append((idx1, idx2))
    
    pair_info.append({
        'pair_id': f'P{len(selected_pairs)}',
        'gal1_id': galaxy1['id'],
        'gal2_id': galaxy2['id'],
        'gal1_ra': galaxy1['ra'],
        'gal1_dec': galaxy1['dec'],
        'gal2_ra': galaxy2['ra'],
        'gal2_dec': galaxy2['dec'],
        'gal1_z': galaxy1['zfinal'],
        'gal2_z': galaxy2['zfinal'],
        'gal1_mass': galaxy1['logm'],
        'gal2_mass': galaxy2['logm'],
        'gal1_sfr': galaxy1['logSFR'],
        'gal2_sfr': galaxy2['logSFR'],
        'separation_kpc': physical_sep_kpc,
        'gal1_delta_ms': galaxy1['DeltaMS'],
        'gal2_delta_ms': galaxy2['DeltaMS']
    })

print(f"Selected {len(selected_pairs)} galaxy pairs meeting all criteria")

# Create DataFrame for easy viewing
if pair_info:
    selected_pairs_df = pd.DataFrame(pair_info)
    print("\nSelected Galaxy Pairs:")
    print(selected_pairs_df[['pair_id', 'gal1_z', 'gal2_z', 'separation_kpc', 'gal1_mass', 'gal2_mass']].to_string(index=False))
else:
    print("No pairs found meeting all criteria")
    selected_pairs_df = pd.DataFrame()

Finding galaxy pairs with optimized algorithm...
Found 18 potential galaxy pairs
Selected 9 galaxy pairs meeting all criteria

Selected Galaxy Pairs:
pair_id  gal1_z  gal2_z  separation_kpc  gal1_mass  gal2_mass
     P1  2.0408  2.0399       49.493738  10.036882   9.980805
     P2  2.0703  2.0703        1.097985  10.498873  10.498873
     P3  1.5059  1.5059        1.261955   9.935256   9.935256
     P4  1.6279  1.6275       33.682392  10.045917   9.846236
     P5  2.2172  2.2168        9.431210  10.001989  10.044949
     P6  2.4593  2.4593        1.642350  10.563625  10.563625
     P7  2.4902  2.4902        1.306919  10.330607  10.330607
     P8  1.6519  1.6518       18.040316  10.048717   9.864955
     P9  1.8031  1.8040       26.776618  10.062596  10.015512


In [ ]:
# Control Sample Selection
print("=== SELECTING CONTROL SAMPLE ===")

def find_isolated_galaxies(catalog, selected_pair_indices, isolation_radius_kpc=100):
    """
    Find isolated galaxies for control sample
    """
    # Get indices of galaxies already in pairs
    paired_indices = set()
    for idx1, idx2 in selected_pair_indices:
        paired_indices.add(idx1)
        paired_indices.add(idx2)
    
    isolated_galaxies = []
    
    # Check each galaxy for isolation (not in pairs)
    for i, galaxy in catalog.iterrows():
        if i in paired_indices:
            continue
            
        # Calculate isolation by checking distances to other galaxies
        is_isolated = True
        
        for j, other_galaxy in catalog.iterrows():
            if i == j or j in paired_indices:
                continue
                
            # Calculate separation
            z_avg = (galaxy['zfinal'] + other_galaxy['zfinal']) / 2
            sep_kpc = calculate_projected_separation_optimized(
                galaxy['ra'], galaxy['dec'],
                other_galaxy['ra'], other_galaxy['dec'],
                z_avg
            )
            
            if sep_kpc <= isolation_radius_kpc:
                is_isolated = False
                break
        
        if is_isolated:
            isolated_galaxies.append(i)
    
    return isolated_galaxies

# Find control sample matching the selected pairs
if len(selected_pairs) > 0:
    print("Finding isolated control galaxies...")
    isolated_indices = find_isolated_galaxies(filtered_catalog, selected_pairs)
    print(f"Found {len(isolated_indices)} isolated galaxy candidates")
    
    # Get mass and redshift ranges from selected pairs
    pair_masses = []
    pair_redshifts = []
    
    for idx1, idx2 in selected_pairs:
        pair_masses.extend([filtered_catalog.iloc[idx1]['logm'], filtered_catalog.iloc[idx2]['logm']])
        pair_redshifts.extend([filtered_catalog.iloc[idx1]['zfinal'], filtered_catalog.iloc[idx2]['zfinal']])
    
    mass_range = (min(pair_masses), max(pair_masses))
    redshift_range = (min(pair_redshifts), max(pair_redshifts))
    
    print(f"Target mass range: {mass_range[0]:.2f} - {mass_range[1]:.2f}")
    print(f"Target redshift range: {redshift_range[0]:.3f} - {redshift_range[1]:.3f}")
    
    # Select control galaxies within these ranges
    control_sample_indices = []
    control_info = []
    
    for isolated_idx in isolated_indices:
        if len(control_sample_indices) >= 10:  # Match number of pairs
            break
        
        control_galaxy = filtered_catalog.iloc[isolated_idx]
        
        # Check if within mass and redshift ranges
        if (mass_range[0] <= control_galaxy['logm'] <= mass_range[1] and 
            redshift_range[0] <= control_galaxy['zfinal'] <= redshift_range[1]):
            
            # Check other criteria (star-forming, size, Hα)
            # Star-forming criterion
            if control_galaxy['DeltaMS'] < 0:
                continue
                
            # Size constraint
            if not pd.isna(control_galaxy['Re_kpc']):
                z = control_galaxy['zfinal']
                angular_diameter_distance = cosmo.angular_diameter_distance(z).to(u.kpc).value
                re_arcsec = (control_galaxy['Re_kpc'] / angular_diameter_distance) * 206265
                if 2 * re_arcsec > 3.0:
                    continue
            
            # Hα observability
            lambda_halpha = 0.6563
            lambda_obs = lambda_halpha * (1 + control_galaxy['zfinal'])
            if not (0.97 <= lambda_obs <= 5.27):
                continue
                
            control_sample_indices.append(isolated_idx)
            control_info.append({
                'control_id': f'C{len(control_sample_indices)}',
                'galaxy_id': control_galaxy['id'],
                'ra': control_galaxy['ra'],
                'dec': control_galaxy['dec'],
                'redshift': control_galaxy['zfinal'],
                'mass': control_galaxy['logm'],
                'sfr': control_galaxy['logSFR'],
                'delta_ms': control_galaxy['DeltaMS']
            })

    print(f"Selected {len(control_sample_indices)} control galaxies meeting all criteria")

    # Create DataFrame for control sample
    if control_info:
        control_sample_df = pd.DataFrame(control_info)
        print("\nSelected Control Galaxies:")
        print(control_sample_df[['control_id', 'redshift', 'mass', 'delta_ms']].to_string(index=False))
    else:
        print("No control galaxies found meeting all criteria")
        control_sample_df = pd.DataFrame()
else:
    print("No pairs selected, skipping control sample selection")
    control_sample_df = pd.DataFrame()

=== SELECTING CONTROL SAMPLE ===
Finding isolated control galaxies...


In [ ]:
# Save Results
print("=== SAVING RESULTS ===")

# Create results directory if it doesn't exist
import os
os.makedirs('../../results/catalogs', exist_ok=True)

# Save selected pairs
if len(selected_pairs) > 0 and 'selected_pairs_df' in locals():
    pairs_file = '../../results/catalogs/selected_pairs.csv'
    selected_pairs_df.to_csv(pairs_file, index=False)
    print(f"✅ Saved {len(selected_pairs)} pairs to selected_pairs.csv")

# Save control sample
if len(control_sample_df) > 0:
    control_file = '../../results/catalogs/control_sample.csv'
    control_sample_df.to_csv(control_file, index=False)
    print(f"✅ Saved {len(control_sample_df)} control galaxies to control_sample.csv")

print(f"\n{'='*50}")
print(f"🎯 FINAL RESULTS:")
print(f"{'='*50}")
print(f"Pairs selected: {len(selected_pairs)}")
print(f"Control galaxies: {len(control_sample_df)}")
print(f"Total galaxies: {len(selected_pairs)*2 + len(control_sample_df)}")